In [2]:
from matplotlib import pyplot
import pandas as pd
import numpy as np
from datetime import datetime
from geopy.distance import vincenty
from geopy.distance import great_circle
import math

In [1]:
train=pd.read_csv('train.csv')
train_fare=train['fare_amount']

# test=pd.read_csv('test.csv')

NameError: name 'pd' is not defined

In [ ]:
[nr,nc]=train.shape

temp=train['tip_amount'].mode()
print 'harsh'
print temp
print 'kara'
# train['tip_amount'].fillna(0.0,inplace=True)
# print train.isnull().sum()




In [ ]:
a=pd.DataFrame(np.ones((3,1)))
a['cv']=[1,3,2]
a['klp']=a['poi']=np.ones(3)
a

In [ ]:
fmt='%Y-%m-%d %H:%M:%S'
# train['pickup_datetime']=train['pickup_datetime'].apply(args=[fmt],func=datetime.strptime)
# print 1
# train['dropoff_datetime']=train['dropoff_datetime'].apply(args=[fmt],func=datetime.strptime)
# print 1
# train['tot_time']=[ (train['dropoff_datetime'][i]-train['pickup_datetime'][i]).total_seconds()  for i in np.arange(nr) ]

# print 1
# train['tot_time']=train['tot_time']/60
# train['tot_time'] = train['tot_time'].round() #rounding to minutes
# train['month']=[ x.month for x in train['pickup_datetime']]
# train['year']=[x.year for x in train]
# train['weekday']=[ x.weekday() for x in train['pickup_datetime']]
# train['hour']=[ x.time().hour for x in train['pickup_datetime']]
train['tot_time']=train['month']=train['year']=train['weekday']=train['hour']=np.ones(nr)

for i in np.arange(nr):
    d1 = datetime.strptime( train['pickup_datetime'][i] , fmt )
    d2 = datetime.strptime( train['dropoff_datetime'][i] , fmt )
    
    if(d2>d1):
        train['tot_time'][i]=round((d2-d1).total_seconds()/60)
    else:
        train['tot_time'][i]=round((d1-d2).total_seconds()/60) #incase data is given wrong, we assume that pick and drop time may have been exchanged
    
    train['month'][i]=d1.month
    train['year'][i]=d1.year
    train['weekday'][i]=d1.weekday()
    train['hour'][i]=d1.time().hour
    
    

In [ ]:
train['tot_dist'] = [ great_circle( (train['pickup_latitude'][i],train['pickup_longitude'][i]) , (train['dropoff_latitude'][i],train['dropoff_longitude'][i])  ).km   for i in np.arange(nr) ]

In [ ]:
useless_cols=['TID','store_and_fwd_flag','fare_amount','dropoff_latitude',''dropoff_longitude]

In [ ]:
fmt='%Y-%m-%d %H:%M:%S'
a=[1,2,3,4,5]
b=['a','b','c','d','e']
for x in np.arange(6):
    print x,'\n'

In [ ]:
useless_cols=['TID','store_and_fwd_flag','fare_amount']